<a href="https://colab.research.google.com/github/akbarpradana/gdp-dashboard/blob/main/Dicoding_TA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#Proyek Analisis Data : E-Commerce Public Dataset


*   Nama        : Akbar Wahyu Pradana
*   Email       : m200b4ky0281@bangkit.academy
*   ID Dicoding : Akbar Wahyu Pradana M200B4KY0281









## Menetukaan Pertanyaan Bisnis



1. 5 State (kota) mana yang memiliki order (pesanan) terbanyak
2. 10 terbanyak order (pesanan) berdasarkan kategori pesananan




## Import Semua Packages/Library yang Digunakan

In [ ]:
!pip install pandas
!pip install plotly
!pip install streamlit -q


import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from google.colab import drive





In [ ]:
!pip install pipreqs

In [ ]:
!wget -q -o - ipv4.icanhazip.com

In [ ]:
! streamlit run app.py &npx localtunnel --port 8501




Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: app.py
Need to install the following packages:
  localtunnel@2.0.2
Ok to proceed? (y) y
your url is: https://plain-spies-tan.loca.lt
^C


NameError: name 'y' is not defined

In [ ]:
drive.mount('/content/gdrive')


Mounted at /content/gdrive


## Data Wrangling

### Gathering Data

In [ ]:
@st.cache_resource
def load_customers_data():
    return pd.read_csv("/content/gdrive/MyDrive/E-Commerce Public Dataset/customers_dataset.csv", sep=',')

@st.cache_resource
def load_products_data():
    return pd.read_csv("/content/gdrive/MyDrive/E-Commerce Public Dataset/products_dataset.csv", sep=',')

@st.cache_resource
def load_orders_data():
    return pd.read_csv("/content/gdrive/MyDrive/E-Commerce Public Dataset/order_items_dataset.csv", sep=',')

In [ ]:
customers_data = load_customers_data()
products_data = load_products_data()
orders_data = load_orders_data()

2024-10-16 06:44:55.825 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 06:44:55.959 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-10-16 06:44:55.962 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 06:44:55.964 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 06:44:56.468 Thread 'Thread-11': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 06:44:56.470 Thread 'Thread-11': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 06:44:57.138 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 06:44:57.139 Thread 'MainThread': miss

###Assesing Data

#### Memeriksa Missing Values

In [ ]:
#CUSTOMER
print("Missing Values pada Customers Dataset")
missing_values_customer = customers_data.isnull().sum()

print(missing_values_customer)

Missing Values pada Customers Dataset
customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64


In [ ]:
#PRODUCTS
print("Missing Values pada Product Dataset")
missing_values_product = products_data.isnull().sum()

print(missing_values_product)

Missing Values pada Product Dataset
product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64


In [ ]:
#Order
print("Missing Values pada Order Dataset")
missing_values_order = orders_data.isnull().sum()

print(missing_values_order)

Missing Values pada Order Dataset
order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64


Disini yang memiliki Missing Values pada product Dataset. kita dapat mengisinya dengan value lainnya, memprediksi atau menghapusnya. Disini saya akan menghapus baris yang memiliki values yang kosong

#### Memeriksa Duplicated Values

In [ ]:
#Customewr
print("Data Duplikat pada Customer Dataset")
duplikat_customer = customers_data.duplicated().sum()

print(duplikat_customer)

Data Duplikat pada Customer Dataset
0


In [ ]:
#Order
print("Data Duplikat pada Order Dataset")
duplikat_order = orders_data.duplicated().sum()

print(duplikat_order)

Data Duplikat pada Order Dataset
0


In [ ]:
#Product
print("Data Duplikat pada Product Dataset")
duplikat_product = products_data.duplicated().sum()

print(duplikat_product)

Data Duplikat pada Product Dataset
0


###Cleaning Data

In [ ]:
#Disini saya hanya cleaning dataset products karena hanya dataset ini yang memiliki missing values
products_data = products_data.dropna()

## Exploratory Data Analysis

In [ ]:
#Judul Dashboard
st.title("Business Orders Dashboard")

2024-10-16 06:44:59.935 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 06:44:59.938 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [ ]:
# 1. 5 kota (state) yang memiliki pesanan (orders) terbanyak
st.header("1. 10 kota (state) yang memiliki pesanan (orders) terbanyak?")

# mengelompokkan kota dan menjumlahkan pesanannya
state_order_counts = customers_data.groupby("customer_state")["customer_id"].count().reset_index()
state_order_counts.columns = ["State", "Number of Orders"]
state_order_counts = state_order_counts.sort_values(by="Number of Orders", ascending=False)



2024-10-16 06:44:59.951 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 06:44:59.953 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [ ]:
# 2. Berapa total harga pesanan per kategori produk?
st.header("2. Berapa total harga pesanan per kategori produk?")

# mennggabungkan orders_data dan products_data
merged_data = pd.merge(orders_data, products_data, on="product_id", how="left")

# menggelompokkan product percategory product dan menjumlahkan harga pesanannya
category_price_totals = merged_data.groupby("product_category_name")["price"].sum().reset_index()
category_price_totals.columns = ["Product Category", "Total Order Price"]
category_price_totals = category_price_totals.sort_values(by="Total Order Price", ascending=False)



2024-10-16 06:45:00.001 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 06:45:00.004 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


##Visualization & Explanatory Analysis

###Pertanyaan 1

In [ ]:
# menampilkan 5 kota yang memiliki pesanan terbanyak
st.subheader("Top States by Number of Orders")
st.write(state_order_counts.head(5))

# menampilkan bar chart
fig_state_orders = px.bar(state_order_counts, x="State", y="Number of Orders",
                          title="State-wise Number of Orders", color="Number of Orders")
st.plotly_chart(fig_state_orders)


2024-10-16 06:45:00.099 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 06:45:00.102 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 06:45:00.156 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 06:45:00.160 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 06:45:00.777 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 06:45:00.779 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 06:45:00.784 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 06:45:00.786 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()

### Pertanyaan 2

In [ ]:

# Menampilkan 10 kategori teratas berdasarkan total harga pesanan
st.subheader("Top 10 Product Categories by Total Order Price")
st.write(category_price_totals.head(10))

# membuat bar chart untuk total harga berdasarkan kategori produk
fig_category_price = px.bar(category_price_totals.head(10), x="Product Category", y="Total Order Price",
                            title="Top 10 Product Categories by Total Order Price", color="Total Order Price")
st.plotly_chart(fig_category_price)

2024-10-16 06:45:00.802 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 06:45:00.804 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 06:45:00.808 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 06:45:00.811 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 06:45:00.857 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 06:45:00.863 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 06:45:00.867 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 06:45:00.870 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()

##Conclusion

1. 5 kota (state) yang memiliki pesanan (orders) terbanyak secara berturut-urut adalah  

<table>
    <tr>
        <th>Kode State</th>
        <th>Total</th>
    </tr>
    <tr>
        <td>SP</td>
        <td>41,746</td>
    </tr>
    <tr>
        <td>RJ</td>
        <td>12,852</td>
    </tr>
    <tr>
        <td>MG</td>
        <td>11,635</td>
    </tr>
    <tr>
        <td>RS</td>
        <td>5,466</td>
    </tr>
    <tr>
        <td>PR</td>
        <td>5,045</td>
    </tr>
</table>


2. Berapa total harga pesanan per kategori produk?  


<table>
    <tr>
        <th>Kategori</th>
        <th>Total Harga</th>
    </tr>
    <tr>
        <td>beleza_saude</td>
        <td>1,258,681.34</td>
    </tr>
    <tr>
        <td>relogios_presentes</td>
        <td>1,205,005.68</td>
    </tr>
    <tr>
        <td>cama_mesa_banho</td>
        <td>1,036,988.68</td>
    </tr>
    <tr>
        <td>esporte_lazer</td>
        <td>988,048.97</td>
    </tr>
    <tr>
        <td>informatica_acessorios</td>
        <td>911,954.32</td>
    </tr>
    <tr>
        <td>moveis_decoracao</td>
        <td>729,762.49</td>
    </tr>
    <tr>
        <td>cool_stuff</td>
        <td>635,290.85</td>
    </tr>
    <tr>
        <td>utilidades_domesticas</td>
        <td>632,248.66</td>
    </tr>
    <tr>
        <td>automotivo</td>
        <td>592,720.11</td>
    </tr>
    <tr>
        <td>ferramentas_jardim</td>
        <td>485,256.46</td>
    </tr>
</table>

In [ ]:
# Navigate to your project folder in Google Drive
%cd /content/drive/MyDrive/path_to_your_project

# Generate the requirements.txt
!pipreqs .


[Errno 2] No such file or directory: '/content/drive/MyDrive/path_to_your_project'
/content
INFO: Not scanning for jupyter notebooks.
INFO: Successfully saved requirements file in ./requirements.txt
